### Installation and Importing Libraries

In [1]:
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import datetime
from plotly.subplots import make_subplots

!pip install ta
import ta

! pip install plotly


/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29413 sha256=ff2822be71b2082bd50c58b1d394c186250ee9977cf69dad029cf140667d8cde
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


### Fetch Data

In [2]:
# Download data from "2023-01-01" to "2024-02-16"
start_date="2023-01-01"
last_date="2024-02-16"
currency_pair = "EURINR=X"
df=yf.download(currency_pair,start=start_date,end=last_date)
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [3]:
# Set the start and end dates for one day and one week
start_date = datetime.datetime(2023,1,1)
last_date=datetime.datetime(2024,2,16)
end_date_one_day = last_date + datetime.timedelta(days=2)
end_date_one_week = last_date + datetime.timedelta(days=7)

# Download data for one day
data_one_day = yf.download(currency_pair, start=start_date, end=end_date_one_day, interval='1d')
data_one_day.reset_index(inplace=True)

# Download data for one week
data_one_week = yf.download(currency_pair, start=start_date, end=end_date_one_week, interval='1d')
data_one_week.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
def plot_candlestick(data):
  candlestick=go.Candlestick(x=data['Date'],
                  open=data['Open'],
                  high=data['High'],
                  low=data['Low'],
                  close=data['Close'])
  fig = go.Figure(data=[candlestick])

  return fig,candlestick

### Grouping data(Month Wise)

#### Analyzing Price Points and Trends

In [5]:
grouped_data=df.groupby(pd.Grouper(key='Date', freq='M')).agg({'Open':'mean','Close':'mean','Low':'mean','High':'mean'}).reset_index()
fg=px.line(grouped_data,x='Date',y=['Open','Close','High','Low'])
fg.show()

In [6]:
_,cand=plot_candlestick(grouped_data)
fig=go.Figure(data=[cand])
fig.show()

Closure

1.   Mostly the trend is bullish except on 31 Jul and 31 Dec 2023. This implies that the pressure on the buyers was greater throughout the year.

2.   From the plot we can observe the condition of Doji on 31 Jul 2023 and 31 Dec 2023 where close and open prices were almost identical implying neither buyers nor sellers succeeded.




### Trading Strategies

In [7]:
class Metrics:

    @staticmethod
    def MovingAverage(data, window, name=""):
        data[name] = data['Close'].rolling(window=window).mean()
        return data

    @staticmethod
    def BollingerBand(data, window=None):
        st = data['Close'].rolling(window=window).std()
        data['upper band'] = data['Close'].rolling(window=window).mean() + (2 * st)
        data['lower band'] = data['Close'].rolling(window=window).mean() - (2 * st)
        return data

    @staticmethod
    def CommodityChannelIndex(data, window):
        typical_price = (data['Close'] + data['High'] + data['Low']) / 3
        data['ma'] = data['Close'].rolling(window=window).mean()
        mean_deviation = typical_price.rolling(window=20).apply(lambda x: np.mean(np.abs(x - np.mean(x))))
        data['cci'] = (typical_price - data['ma']) / (0.015 * mean_deviation)
        return data

    @staticmethod
    def Macd(data, MACD_SLOW_EMA=12, MACD_FAST_EMA=26, MACD_SIGNAL_PERIOD=9):
        macd=ta.trend.MACD(data['Close'], window_slow= MACD_SLOW_EMA, window_fast= MACD_FAST_EMA, window_sign = MACD_SIGNAL_PERIOD)
        data["macd"],data["signal"],data["hist"]=macd.macd(), macd.macd_signal(), macd.macd_diff()
        return data

    @staticmethod
    def RSI(data, window):
        rsi_indicator = ta.momentum.RSIIndicator(data['Close'], window=window)
        rsi_values = rsi_indicator.rsi()
        data['rsi'] = rsi_values
        return data

In [8]:
def trading_tools(df):
  met=Metrics.MovingAverage(df,10,'ma_20')
  met=Metrics.BollingerBand(df,window=20)
  met=Metrics.CommodityChannelIndex(df,window=20)
  met=Metrics.Macd(df)
  met=Metrics.RSI(df,window=20)
  return met

In [9]:
trading_stra_one_day=trading_tools(data_one_day)
trading_stra_one_week=trading_tools(data_one_week)

In [10]:
data_one_day=data_one_day.iloc[:,:7]
data_one_week=data_one_week.iloc[:,:7]

#### Plots

In [11]:
def plot_cci(data):

  _,candlestick=plot_candlestick(data)

  cci=go.Scatter(x=data['Date'],y=data['cci'],name="CCI with {} window size",line={'color':'brown'})

  fig = go.Figure(data=[candlestick,cci])
  fig.layout.xaxis.type = 'category'
  fig.show()



In [12]:
def plot_bb(data):

  # bb=Metrics.BollingerBand(data,20)
  ma_20=Metrics.MovingAverage(data,20,'ma_20')

  _,candlestick=plot_candlestick(data)

  ma_ub=go.Scatter(x=data['Date'],y=data['upper band'],name="upper band",line={'color':'red'})
  ma=go.Scatter(x=data['Date'],y=data['ma_20'],name="Moving Average with 20 window size",line={'color':'grey'})
  ma_lb=go.Scatter(x=data['Date'],y=data['lower band'],name="lower band",line={'color':'brown'})

  fig = go.Figure(data=[candlestick,ma_ub,ma,ma_lb])
  fig.layout.xaxis.type = 'category'
  fig.show()



In [13]:
def plot_ma(data,window,color):
  data=Metrics.MovingAverage(data,window,'ma_{}'.format(window))

  _,candlestick=plot_candlestick(data)
  nami='ma_{}'.format(window)
  ma=go.Scatter(x=data['Date'],y=data[nami],name="Moving Average with {} window size".format(window),line={'color':color})

  return ma



In [14]:
def plot_macd(stock_df, macd, signal, hist):
    fig = make_subplots( rows=2, cols=1)

    candlestick = go.Candlestick(x=stock_df['Date'],
                                 open=stock_df['Open'],
                                 high=stock_df['High'],
                                 low=stock_df['Low'],
                                 close=stock_df['Close'], name = 'daily candle')

    positive_hist = hist[hist>0]
    negative_hist = hist[hist<0]

    macd_line = go.Scatter(x=stock_df['Date'], y=macd, name='MACD', line_color='blue')
    signal_line = go.Scatter(x=stock_df['Date'], y=signal, name='Signal', line_color='orange')
    pos_hist_bar = go.Bar(x=stock_df['Date'], y=positive_hist, name='Positive Hist', marker={'color': 'green'})
    neg_hist_bar = go.Bar(x=stock_df['Date'], y=negative_hist, name='Negative Hist', marker={'color': 'red'})

    fig.add_trace(candlestick, row=1,col=1)
    fig.add_trace(macd_line, row=2, col=1)
    fig.add_trace(signal_line, row=2, col=1)
    fig.add_trace(pos_hist_bar, row=2, col=1)
    fig.add_trace(neg_hist_bar, row=2, col=1)

    fig.update_layout(title_text=f'MACD', title_x=0.5,
                     xaxis_type = 'category',xaxis_rangeslider_visible = False,
                     xaxis_showticklabels = False,
                     xaxis2_type = 'category' )
    fig.show()

#### One day and One week from 16th feb 2024


##### One Day

In [15]:
one_day=trading_stra_one_day

In [16]:
_,candlestick=plot_candlestick(one_day)
fig_20=plot_ma(one_day,20,"red")
fig_50=plot_ma(one_day,50,"blue")
fig = go.Figure(data=[candlestick,fig_20,fig_50])
fig.layout.xaxis.type = 'category'
fig.show()

In [17]:
MACD_FAST_EMA = 12#default
MACD_SLOW_EMA = 26#default
MACD_SIGNAL_PERIOD = 9#default
macd=ta.trend.MACD(one_day['Close'], window_slow= MACD_SLOW_EMA, window_fast= MACD_FAST_EMA, window_sign = MACD_SIGNAL_PERIOD)
plot_macd(one_day, macd.macd(), macd.macd_signal(), macd.macd_diff())

In [18]:
plot_bb(one_day)

In [19]:
plot_cci(one_day)

**Closure**

*  *Moving Average*
1. The time phase **2023-08-30** to **2023-11-10** generally exhibits the **Buying** signal, where [`Current Close<MovingAverage(20)<MovingAverage(50)`].




---


*  *Moving Average Convergence Divergence*(MACD)
1. From **2023-12-04** and after, it exhibits the **Selling** signal, whereas the period from **2023-10-06** exibits **Buying** signal.

2. Keeping in Mind
```
When the MACD line crosses the signal line from below → BUY signal
When the MACD line crosses the signal line from above → SELL signal.
```


---

* *Bollinger Band*(BB)
1. From **2023-12-04** onwards, it exhibits the **Selling** signal, whereas the period from **2023-10-06** onwards exibits **Buying** signal.



---


* *Commodity Channel Index*(BB)
Overbought and Oversold Conditions:

1. CCI values above +100 are considered overbought, Indicating that the price is unusually high compared to its average.
CCI values below -100 are considered oversold, indicating that the price is unusually low compared to its average.


2. Crossing the +100 level from below may signal a potential uptrend or bullish reversal.
Crossing the -100 level from above may signal a potential downtrend or bearish reversal.





---









##### One Week

In [20]:
one_week=trading_stra_one_week

In [21]:
_,candlestick=plot_candlestick(one_week)
fig_20=plot_ma(one_week,20,"red")
fig_50=plot_ma(one_week,50,"blue")
fig = go.Figure(data=[candlestick,fig_20,fig_50])
fig.layout.xaxis.type = 'category'
fig.show()

In [22]:
MACD_FAST_EMA = 12#default
MACD_SLOW_EMA = 26#default
MACD_SIGNAL_PERIOD = 9#default
macd=ta.trend.MACD(one_week['Close'], window_slow= MACD_SLOW_EMA, window_fast= MACD_FAST_EMA, window_sign = MACD_SIGNAL_PERIOD)
plot_macd(one_week, macd.macd(), macd.macd_signal(), macd.macd_diff())

In [23]:
plot_bb(one_week)

In [24]:
plot_cci(one_week)

**Closure**

*  *Moving Average*
1. The time phase **2023-08-31** onwardso ans **2024-01-30** onwards generally exhibits the **Buying** signal, where [`Current Close<MovingAverage(20)<MovingAverage(50)`].




---


*  *Moving Average Convergence Divergence*(MACD)
1. From **2023-05-01** and ** 2023-07-25** onwards, it exhibits the **Selling** signal, whereas from **2023-06-09** and **2023-03-14** it exibits **Buying** signal.

2. Keeping in Mind
```
When the MACD line crosses the signal line from below → BUY signal
When the MACD line crosses the signal line from above → SELL signal.
```


---

* *Bollinger Band*(BB)
1. The plot shows high volatility throughout the given time phase.



---











### Decision Strategies and Indicators


In [25]:
# buy/sell indicator using moving average,macd

class Indicator:

  def moving_average_indicator(self,df_ind):

    indicator="Hold"

    colm=[df_ind['Close'],df_ind['2_ma'],df_ind['4_ma']]

    if(colm==sorted(colm)):
      indicator="Buy"

    if(colm==sorted(colm,reverse=True)):
      indicator="Sell"

    return indicator

  def moving_average_strategy(self,df_ind,window:list,name=""):
    df_ind=Metrics.MovingAverage(df_ind,window=window[0],name="2_ma")#2

    df_ind=Metrics.MovingAverage(df_ind,window=window[1],name="4_ma")#4

    df_ind['ma_ind']=df_ind.apply(lambda x: self.moving_average_indicator(x),axis=1)

    return df_ind

  def macd_indicator(self,data):
    last_hist = data['hist']

    prev_hist = data['prev_hist']

    indicator = 'HOLD'

    # Provide BUY or SELL indications during crossovers
    if not np.isnan(prev_hist) and not np.isnan(last_hist):

        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            indicator = 'BUY' if last_hist > 0 else 'SELL'

    return indicator

  def apply_macd_strategy(self,stock_df):

    stock_df= Metrics.Macd(stock_df,5,10,9)

    stock_df['prev_hist'] = stock_df['hist'].shift(1)

    stock_df['macd_indicator'] = stock_df.apply(lambda row: self.macd_indicator(row), axis=1)

    return stock_df

  def apply_trading_algo(self,stock_df):

    CHECK_RSI_LAST_SESSIONS = 3
    RSI_LOWER_LIMIT = 20
    RSI_UPPER_LIMIT = 40

    stock_df=self.moving_average_strategy(stock_df,window=[2,4])
    stock_df=self.apply_macd_strategy(stock_df)
    stock_df['indicator'] = 'HOLD'
    buy_sell_signals = stock_df[(stock_df['macd_indicator'] != 'HOLD') | (stock_df['ma_ind'] != 'HOLD')]

    for index,row in buy_sell_signals.iterrows():
        analysis_df = stock_df.loc[:index].tail(3)
        macd_ind = analysis_df.iloc[-1]['macd_indicator']
        ma_ind = analysis_df.iloc[-1]['ma_ind']
        current_indicator = 'HOLD'
        current_indicator = macd_ind if macd_ind != 'HOLD' else ma_ind
        if current_indicator != 'HOLD':
            if (current_indicator == 'BUY' and analysis_df['rsi'].min() <= RSI_LOWER_LIMIT) \
                    or (current_indicator == 'SELL' and analysis_df['rsi'].max() >= RSI_UPPER_LIMIT):
                stock_df.at[index, 'indicator'] = current_indicator

    return stock_df


#### Result

In [26]:
ind=Indicator()
# res1=ind.apply_trading_algo(data_one_week)

In [27]:
def plot_buy_sell_chart( stock):
    _,candlestick = plot_candlestick(stock)

    fig = go.Figure(data=[candlestick],
                   layout=go.Layout(title=go.layout.Title(text=f"BUY/SELL RECOMMENDATIONS - FOR EUR/INR")))
    fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False

    for index, row in stock.iterrows():
        #print(row)
        if (row['indicator'] != 'HOLD'):
            line_colour = 'orange' if (row['indicator'] == 'BUY') else 'purple'
            fig.add_vline(x=row.name, line_width=3, line_dash="dash", line_color=line_colour)


    fig.show()


In [28]:
def my_trading_algo(stock):

    # Apply the strategies individual
    stock = trading_tools(stock)


    # Merge the strategies to create a final BUY/SELL/HOLD indicator

    stock =ind. apply_trading_algo(stock)

    # Plot the buy/sell recommendations on 2021 data
    plot_buy_sell_chart(stock)

    return stock

In [29]:
final1=my_trading_algo(one_day)
final1

,Date,Open,High,Low,Close,Adj Close,Volume,ma_20,upper band,lower band,...,signal,hist,rsi,ma_50,2_ma,4_ma,ma_ind,prev_hist,macd_indicator,indicator
0,2023-01-02,87.157700,88.400002,87.157700,88.482002,88.482002,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sell,NaN,HOLD,HOLD
1,2023-01-03,88.292000,88.313004,87.225998,88.300003,88.300003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,88.391003,NaN,Buy,NaN,HOLD,HOLD
2,2023-01-04,87.219002,87.922997,87.219002,87.219002,87.219002,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.759502,NaN,Buy,NaN,HOLD,HOLD
3,2023-01-05,87.580002,87.807999,86.757004,87.605003,87.605003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.412003,87.901503,Hold,NaN,HOLD,HOLD
4,2023-01-06,86.819000,87.614998,86.647003,86.827003,86.827003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.216003,87.487753,Buy,NaN,HOLD,HOLD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,2024-02-12,88.418404,89.594002,88.418404,89.444000,89.444000,0,89.549921,90.872856,88.915585,...,0.180902,-0.030290,45.141747,90.377452,89.361000,89.346001,Sell,0.008016,SELL,SELL
291,2024-02-13,89.358002,89.511002,88.843002,89.309998,89.309998,0,89.478420,90.771312,88.883328,...,0.172074,-0.035311,44.205547,90.357612,89.376999,89.358250,Hold,-0.030290,HOLD,HOLD
292,2024-02-14,88.862000,89.032997,88.733002,88.862000,88.862000,0,89.359720,90.762478,88.747062,...,0.175419,0.013380,41.198690,90.337612,89.085999,89.223499,Buy,-0.035311,BUY,HOLD
293,2024-02-15,88.987000,89.440002,88.930000,89.036003,89.036003,0,89.301521,90.686469,88.682471,...,0.176210,0.003165,42.789685,90.324992,88.949001,89.163000,Hold,0.013380,HOLD,HOLD


In [30]:
# table of indicators
table1=final1[['Date', 'Open', 'High', 'Low', 'Close', 'ma_ind',  'macd_indicator']]

table1=table1.iloc[-1]
pd.DataFrame(table1).reset_index()

,index,294
0,Date,2024-02-16 00:00:00
1,Open,89.347
2,High,89.411003
3,Low,89.046997
4,Close,89.357002
5,ma_ind,Sell
6,macd_indicator,SELL


In [31]:
final2=my_trading_algo(one_week)
final2

,Date,Open,High,Low,Close,Adj Close,Volume,ma_20,upper band,lower band,...,signal,hist,rsi,ma_50,2_ma,4_ma,ma_ind,prev_hist,macd_indicator,indicator
0,2023-01-02,87.157700,88.400002,87.157700,88.482002,88.482002,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Sell,NaN,HOLD,HOLD
1,2023-01-03,88.292000,88.313004,87.225998,88.300003,88.300003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,88.391003,NaN,Buy,NaN,HOLD,HOLD
2,2023-01-04,87.219002,87.922997,87.219002,87.219002,87.219002,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.759502,NaN,Buy,NaN,HOLD,HOLD
3,2023-01-05,87.580002,87.807999,86.757004,87.605003,87.605003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.412003,87.901503,Hold,NaN,HOLD,HOLD
4,2023-01-06,86.819000,87.614998,86.647003,86.827003,86.827003,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.216003,87.487753,Buy,NaN,HOLD,HOLD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2024-02-16,89.347000,89.411003,89.046997,89.357002,89.357002,0,89.228821,90.587914,88.678127,...,0.163913,-0.049186,45.645558,90.314692,89.196503,89.141251,Sell,0.003165,SELL,SELL
295,2024-02-19,88.461304,89.460999,88.461304,89.377998,89.377998,0,89.244801,90.587873,88.672248,...,0.144997,-0.075666,45.831733,90.324062,89.367500,89.158251,Sell,-0.049186,HOLD,HOLD
296,2024-02-20,89.402000,89.790001,89.254997,89.403000,89.403000,0,89.273000,90.492372,88.677448,...,0.123313,-0.086734,46.063298,90.319522,89.390499,89.293501,Sell,-0.075666,HOLD,HOLD
297,2024-02-21,89.473999,89.625000,89.420998,89.477997,89.477997,0,89.294700,90.391110,88.698210,...,0.099677,-0.094543,46.781662,90.309722,89.440498,89.403999,Sell,-0.086734,HOLD,HOLD


In [35]:
# table of indicators
# table2=final2[['Date', 'Open', 'High', 'Low', 'Close', 'ma_ind',  'macd_indicator']]

# table2=table1[(table1['Date']>'2024-02-16' )& (table1['Date']<='2024-02-22' )]
# table2

table2=final2[['Date', 'Open', 'High', 'Low', 'Close', 'ma_ind',  'macd_indicator']]

table2=table2[(table2['Date']>'2024-02-16' )& (table2['Date']<='2024-02-22' )]
pd.DataFrame(table2).reset_index()

,index,Date,Open,High,Low,Close,ma_ind,macd_indicator
0,295,2024-02-19,88.461304,89.460999,88.461304,89.377998,Sell,HOLD
1,296,2024-02-20,89.402000,89.790001,89.254997,89.403000,Sell,HOLD
2,297,2024-02-21,89.473999,89.625000,89.420998,89.477997,Sell,HOLD
3,298,2024-02-22,89.682999,90.143997,89.475998,89.675003,Sell,HOLD


Closure

1. From the above decision we can say that the ideal time for buying stock was from 2023-10-12 to 2023-10-23, whereas for selling it was 2023-12-29.

2. Here are some potential reasons for the resultant Indicator:

  Conflicting Signals: There might be other indicators or factors in our trading strategy that are causing the final indicator.

  Priority of Signals: If there are multiple indicators or criteria for generating buy/sell/hold signals, ensure that the priority of these signals is correctly defined.

  Insufficient Data: Insufficient data can lead to unreliable signals.

Assumptions -
1. In the apply_trading_algo function, I'm implementing a trading algorithm that combines signals from moving averages (MA) and Moving Average Convergence Divergence (MACD) indicators.

2. Combining Signals: I'm combining signals from both MACD and MA indicators to make trading decisions. If either indicator provides a 'BUY' or 'SELL' signal, you consider taking action.

3. RSI Confirmation: I'm using the Relative Strength Index (RSI) to confirm the buy/sell signals. If the current indicator is 'BUY' and the RSI is below a certain threshold, or if the indicator is 'SELL' and the RSI is above a certain threshold, you execute the trade.

4. No patterns were observed for engulfing and Morning/Evening trend.

5. In our case I've taken 20 as window size.



To optimize this function and ensure clearer indications, we can consider the following:

1. Refinement of Trading Rules: Review and refine your trading rules based on historical data analysis and backtesting. Ensuring that the thresholds for RSI and other parameters are chosen appropriately to minimize false signals and improve the accuracy of the trading strategy.

2. Consider Additional Confirmation Signals: We can use additional technical indicators or fundamental factors that can provide confirmation for buy/sell signals generated by MA and MACD. This can enhance the reliability of the trading strategy.

3. Monitoring and Adaptation: Continuously monitoring the performance of our trading strategy in real-time and adapting it as needed based on changing market conditions, emerging trends, and new information.